### ニューラルネットワーク回帰　パラメータチューニング

In [1]:
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55369, 13)

In [4]:
train_pkl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55369 entries, 0 to 55582
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   55369 non-null  int64  
 1   accommodates                         55369 non-null  int8   
 2   bathrooms                            55369 non-null  int8   
 3   bedrooms                             55369 non-null  int8   
 4   host_response_rate                   55369 non-null  int8   
 5   number_of_reviews                    55369 non-null  int8   
 6   y                                    55369 non-null  float64
 7   cancellation_policy_moderate         55369 non-null  uint8  
 8   cancellation_policy_strict           55369 non-null  uint8  
 9   cancellation_policy_super_strict_30  55369 non-null  uint8  
 10  cancellation_policy_super_strict_60  55369 non-null  uint8  
 11  room_type_Private room      

In [5]:
train_pkl.isnull().all()

id                                     False
accommodates                           False
bathrooms                              False
bedrooms                               False
host_response_rate                     False
number_of_reviews                      False
y                                      False
cancellation_policy_moderate           False
cancellation_policy_strict             False
cancellation_policy_super_strict_30    False
cancellation_policy_super_strict_60    False
room_type_Private room                 False
room_type_Shared room                  False
dtype: bool

In [6]:
np.max(train_pkl.id), np.max(train_pkl.y)

(55582, 1998.9999999999993)

In [7]:
np.min(train_pkl.id), np.min(train_pkl.y)

(0, 1.0)

In [8]:
train_pkl['y'] = train_pkl['y'].astype('float32')

## 訓練データとテストデータに分割

In [9]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [10]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

In [11]:
grid_param = [
    {
    'solver': ['sgd'],    
    'activation': ['logistic', 'relu'],
    'hidden_layer_sizes':[3, 5, 6, 7, 10, 15],
#     'learning_rate_init': [0.001, 0.01, 0.1, 1],
    'alpha': [0.1, 0.01, 0.001, 0.0001],
#     'max_iter': [],
    'random_state': [42]
    },
    {
    'solver': ['adam'],    
    'activation': ['logistic', 'relu'],
    'hidden_layer_sizes':[3, 5, 6, 7, 10, 15],
#     'learning_rate_init': [0.001, 0.01, 0.1, 1],
    'alpha': [0.1, 0.01, 0.001, 0.0001],
#     'max_iter': [],
    'random_state': [42]
    }
]


### グリッドサーチ

In [12]:
gs = GridSearchCV(estimator=MLPRegressor(), param_grid=grid_param, scoring=make_scorer(mean_squared_error, greater_is_better=False), cv=5, return_train_score=False)

In [13]:
gs.fit(train_X, train_y)

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:151: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:151: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### 一番良かったパラメータで学習

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

In [ ]:
# gs.cv_results_['mean_test_score']

In [ ]:
NN = MLPRegressor(**gs.best_params_)

In [ ]:
NN = NN.fit(train_X, train_y)

### テストデータで学習

In [ ]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [ ]:
test_x.shape, test_y.shape

In [ ]:
pred_y = NN.predict(test_x)

In [ ]:
np.sqrt(mean_squared_error(test_y, pred_y))

### 検証データで学習

In [ ]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3'

In [ ]:
valid.shape

In [ ]:
# ID の保存
valid_pass = valid.id.values

In [ ]:
valid_X = valid.iloc[:, 1:].values

In [ ]:
valid_X.shape, train_X.shape

In [ ]:
pred_valid_y = NN.predict(valid_X)

In [ ]:
pred_valid_y.shape

In [ ]:
type(valid_pass), type(pred_valid_y)

In [ ]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [ ]:
result_df.to_csv("./NN_4.csv", header=False)